# Обучение и оценка мульти-входовой нейронной сети

In [44]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm 

# Импортируем нашу архитектуру модели из соседнего файла.
from multi_input_model import MultiInputModel

### Шаг 1: Конфигурация

ключевые гиперпараметры и настройки

In [45]:
# Список имен тестов в том порядке, в котором мы будем их загружать и подавать в модель.
TEST_NAMES = ["T1back", "TStroop", "T258", "T274", "T278"]
# Размер батча - количество примеров, обрабатываемых за один шаг обучения.
BATCH_SIZE = 32
# Скорость обучения (learning rate) - шаг, с которым модель обновляет свои веса.
LEARNING_RATE = 1e-4 
# Максимальное количество эпох обучения.
NUM_EPOCHS = 150
# "Терпение" для механизма ранней остановки. Если ошибка на валидации не улучшается
# в течение `PATIENCE` эпох, обучение прекращается.
PATIENCE = 10 

# Важно: этот параметр должен совпадать с STROOP_PROCESSING_WAY в prepare_multi_test_data.py
STROOP_PROCESSING_WAY = 2

### Шаг 2: Загрузка данных
Здесь же происходит специальная обработка для теста Струпа, если был выбран способ подготовки `STROOP_PROCESSING_WAY = 1`.

In [46]:
print("Загрузка подготовленных данных...")
# Загружаем 5 массивов с данными тестов в список `Xs`.
Xs = []
for name in TEST_NAMES:
    data = np.load(f"X_{name}.npy")
    # Специальная обработка для TStroop, если он был сохранен как 4D тензор
    if name == "TStroop" and STROOP_PROCESSING_WAY == 1:
        # Преобразуем (batch_size, num_subtests, max_len_subtest, num_features) в
        # (batch_size, num_subtests * max_len_subtest, num_features)
        batch_size, num_subtests, max_len_subtest, num_features = data.shape
        data = data.reshape(batch_size, num_subtests * max_len_subtest, num_features)
        print(f"Данные TStroop преобразованы из 4D в 3D с формой: {data.shape}")
    Xs.append(data)
y = np.load("y_aligned.npy")

print("Данные успешно загружены.")

Загрузка подготовленных данных...
Данные успешно загружены.


### Шаг 3: Разделение данных, Масштабирование и создание DataLoader'ов



In [47]:
# Константа для значения паддинга, чтобы не использовать магическое число
FILLING_VALUE = -1.0

# 1. Разделение индексов
indices = list(range(len(y)))
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_indices, test_size=0.25, random_state=42)

# 2. Обучение скейлеров и масштабирование данных
print("Масштабирование данных...")
scalers = [StandardScaler() for _ in TEST_NAMES]
Xs_scaled = [x.copy() for x in Xs] # Создаем копию данных для масштабирования

for i, (test_name, X_test) in enumerate(zip(TEST_NAMES, Xs)):
    # Выбираем данные для обучения скейлера
    train_data = X_test[train_indices]
    
    # Преобразуем 3D данные в 2D (n_samples * sequence_length, n_features)
    n_samples, seq_len, n_features = train_data.shape
    reshaped_train_data = train_data.reshape(-1, n_features)

    # Обучаем скейлер только на "активных" данных (не на паддинге)
    # Мы предполагаем, что если первый признак - это FILLING_VALUE, то вся строка является паддингом
    active_mask = reshaped_train_data[:, 0] != FILLING_VALUE
    scalers[i].fit(reshaped_train_data[active_mask])
    
    # Теперь применяем обученный скейлер ко всем данным (train, val, test)
    n_samples_total, seq_len_total, _ = X_test.shape
    reshaped_total_data = X_test.reshape(-1, n_features)
    
    total_active_mask = reshaped_total_data[:, 0] != FILLING_VALUE
    reshaped_total_data[total_active_mask] = scalers[i].transform(reshaped_total_data[total_active_mask])
    
    # Возвращаем данные в исходный 3D формат и сохраняем в Xs_scaled
    Xs_scaled[i] = reshaped_total_data.reshape(n_samples_total, seq_len_total, n_features)

print("Масштабирование завершено.")

# 3. Создание Dataset и DataLoader
class MultiInputDataset(Dataset):
    def __init__(self, xs_list, y_arr):
        self.xs = [torch.tensor(x, dtype=torch.float32) for x in xs_list]
        self.y = torch.tensor(y_arr, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return [x[idx] for x in self.xs], self.y[idx]

def create_subset(indices_list):
    # Используем масштабированные данные Xs_scaled!
    subset_xs = [x[indices_list] for x in Xs_scaled]
    subset_y = y[indices_list]
    return MultiInputDataset(subset_xs, subset_y)

print("Создание загрузчиков данных (DataLoader)...")
train_dataset = create_subset(train_indices)
val_dataset = create_subset(val_indices)
test_dataset = create_subset(test_indices)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Загрузчики данных готовы.")

Масштабирование данных...
Масштабирование завершено.
Создание загрузчиков данных (DataLoader)...
Загрузчики данных готовы.


### Шаг 4: Инициализация модели, функции потерь и оптимизатора

In [48]:
# Собираем словарь с количеством признаков для каждого теста. Это нужно для инициализации модели.
input_dims = {name: Xs[i].shape[2] for i, name in enumerate(TEST_NAMES)}

model = MultiInputModel(input_dims=input_dims)
# Функция потерь (Loss Function). L1Loss - это MAE.
criterion = nn.L1Loss() 
# Оптимизатор. Adam - один из самых популярных и эффективных алгоритмов оптимизации.
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("Модель инициализирована:")
print(model)

Модель инициализирована:
MultiInputModel(
  (branches): ModuleDict(
    (T1back): SubtestBranch(
      (lstm): LSTM(13, 32, batch_first=True)
      (fc): Linear(in_features=32, out_features=16, bias=True)
      (relu): ReLU()
    )
    (TStroop): SubtestBranch(
      (lstm): LSTM(5, 32, batch_first=True)
      (fc): Linear(in_features=32, out_features=16, bias=True)
      (relu): ReLU()
    )
    (T258): SubtestBranch(
      (lstm): LSTM(10, 32, batch_first=True)
      (fc): Linear(in_features=32, out_features=16, bias=True)
      (relu): ReLU()
    )
    (T274): SubtestBranch(
      (lstm): LSTM(13, 32, batch_first=True)
      (fc): Linear(in_features=32, out_features=16, bias=True)
      (relu): ReLU()
    )
    (T278): SubtestBranch(
      (lstm): LSTM(13, 32, batch_first=True)
      (fc): Linear(in_features=32, out_features=16, bias=True)
      (relu): ReLU()
    )
  )
  (head): Sequential(
    (0): Linear(in_features=80, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropou

### Шаг 5: Цикл обучения

Запускаем основной цикл, в котором модель будет итеративно обучаться на данных, проходить валидацию и сохранять свою лучшую версию.

In [49]:
best_val_loss = float('inf') # Начальное значение лучшей ошибки на валидации (бесконечность).
patience_counter = 0 # Счетчик для ранней остановки.

print(f"Начало обучения на {NUM_EPOCHS} эпох...")
for epoch in range(NUM_EPOCHS):
    # --- Фаза обучения (Training) --- 
    model.train() # Переводим модель в режим обучения.
    train_loss = 0
    # tqdm - обертка для `train_loader` для отображения красивого progress bar'а.
    for x_batch, y_batch in tqdm(train_loader, desc=f"Эпоха {epoch+1}/{NUM_EPOCHS} [Обучение]"):
        optimizer.zero_grad() # Обнуляем градиенты с предыдущего шага.
        y_pred = model(x_batch).squeeze() # Делаем предсказание и убираем лишние размерности.
        loss = criterion(y_pred, y_batch) # Считаем ошибку.
        loss.backward() # Вычисляем градиенты (обратное распространение ошибки).
        optimizer.step() # Обновляем веса модели.
        train_loss += loss.item() # Суммируем ошибку.
    
    avg_train_loss = train_loss / len(train_loader)

    # --- Фаза валидации (Validation) --- 
    model.eval() # Переводим модель в режим оценки (отключаются Dropout и т.д.).
    val_preds = []
    val_targets = []
    with torch.no_grad(): # В этом блоке градиенты не вычисляются для экономии ресурсов.
        for x_batch, y_batch in val_loader:
            y_pred = model(x_batch).squeeze()
            val_preds.append(y_pred.cpu().numpy()) # Собираем предсказания
            val_targets.append(y_batch.cpu().numpy()) # и реальные значения.
            
    val_preds = np.concatenate(val_preds)
    val_targets = np.concatenate(val_targets)
    # Считаем среднюю абсолютную ошибку (MAE) на валидационной выборке.
    avg_val_loss = mean_absolute_error(val_targets, val_preds)
    print(f"Эпоха [{epoch+1}/{NUM_EPOCHS}] | Ошибка на обучении: {avg_train_loss:.4f} | Ошибка на валидации (MAE): {avg_val_loss:.4f}")

    # --- Ранняя остановка и сохранение лучшей модели ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        # Сохраняем состояние модели (ее веса), если она показала лучший результат.
        torch.save(model.state_dict(), 'best_multi_input_model.pth')
        print(f"Ошибка на валидации улучшилась. Модель сохранена в 'best_multi_input_model.pth'")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"Ранняя остановка: ошибка не улучшалась {patience_counter} эпох.")
            break

Начало обучения на 150 эпох...

















Эпоха 1/150 [Обучение]: 100%|██████████| 92/92 [00:01<00:00, 58.68it/s]


Эпоха [1/150] | Ошибка на обучении: 43.9815 | Ошибка на валидации (MAE): 44.3726
Ошибка на валидации улучшилась. Модель сохранена в 'best_multi_input_model.pth'


















Эпоха 2/150 [Обучение]: 100%|██████████| 92/92 [00:01<00:00, 55.59it/s]


Эпоха [2/150] | Ошибка на обучении: 42.4779 | Ошибка на валидации (MAE): 39.6439
Ошибка на валидации улучшилась. Модель сохранена в 'best_multi_input_model.pth'


...

Эпоха [68/150] | Ошибка на обучении: 9.3187 | Ошибка на валидации (MAE): 8.8144
















Эпоха 69/150 [Обучение]: 100%|██████████| 92/92 [00:01<00:00, 58.35it/s]


Эпоха [69/150] | Ошибка на обучении: 9.3392 | Ошибка на валидации (MAE): 8.6616
















Эпоха 70/150 [Обучение]: 100%|██████████| 92/92 [00:01<00:00, 61.57it/s]


Эпоха [70/150] | Ошибка на обучении: 9.3843 | Ошибка на валидации (MAE): 8.7405
Ранняя остановка: ошибка не улучшалась 10 эпох.


### Шаг 6: Финальная оценка на тестовой выборке



In [51]:
print("\n--- Тестирование ---")
# Загружаем веса лучшей модели, сохраненной ранее.
model.load_state_dict(torch.load('best_multi_input_model.pth'))
model.eval() # Переводим в режим оценки.

test_preds = []
test_targets = []
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader, desc="[Тест]"):
        y_pred = model(x_batch).squeeze()
        test_preds.append(y_pred.cpu().numpy())
        test_targets.append(y_batch.cpu().numpy())

test_preds = np.concatenate(test_preds)
test_targets = np.concatenate(test_targets)

# Считаем и выводим финальные метрики на данных, которые модель еще не видела.
test_mse = mean_squared_error(test_targets, test_preds)
test_mae = mean_absolute_error(test_targets, test_preds)

print("\nИтоговые результаты на тестовой выборке:")
print(f"  Средняя квадратичная ошибка (MSE): {test_mse:.4f}")
print(f"  Средняя абсолютная ошибка (MAE): {test_mae:.4f}")


--- Тестирование ---




[Тест]: 100%|██████████| 31/31 [00:00<00:00, 160.40it/s]


Итоговые результаты на тестовой выборке:
  Средняя квадратичная ошибка (MSE): 118.7586
  Средняя абсолютная ошибка (MAE): 8.6728
